# [INFER] SIIM-COV19 efnb7 (study+2class)

thanks to https://www.kaggle.com/xhlulu/siim-covid-19-convert-to-jpg-256px  
thanks to https://www.kaggle.com/awsaf49/vinbigdata-cxr-ad-yolov5-14-class-infer  
train_study: https://www.kaggle.com/h053473666/siim-covid19-efnb7-train-study  
train_image: https://www.kaggle.com/h053473666/siim-cov19-yolov5-train  
train_2class: https://www.kaggle.com/h053473666/siim-covid19-efnb7-train-fold0-5-2class  
  
version1:Original hyperparameters (yolov5)  
version4:New hyperparameters (yolov5)  

* v35
    1. study : 
    2. 2 class : 
    3. image : yolo v5x 512 epohs 50 kfold

In [1]:
!conda install '/kaggle/input/pydicom-conda-helper/libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/libgcc-ng-9.3.0-h2828fa1_19.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/conda-4.10.1-py37h89c1867_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/certifi-2020.12.5-py37h89c1867_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/openssl-1.1.1k-h7f98852_0.tar.bz2' -c conda-forge -y
!pip install /kaggle/input/weightedboxesfusion


######################################################################## | 100% 
Preparing transaction: - done
Verifying transaction: | done
Executing transaction: - \ done

######################################################################## | 100% 
Preparing transaction: - done
Verifying transaction: | done
Executing transaction: - \ | done

######################################################################## | 100% 
Preparing transaction: - done
Verifying transaction: | done
Executing transaction: - done

######################################################################## | 100% 
Preparing transaction: - done
Verifying transaction: | done
Executing transaction: - \ | / - \ | / - \ | / - \ | / - \ | / done

######################################################################## | 100% 
Preparing transaction: - done
Verifying transaction: | done
Executing transaction: - \ done

#################

In [2]:
import os

from PIL import Image
import pandas as pd
from tqdm.auto import tqdm

import numpy as np, pandas as pd
from glob import glob
import shutil, os
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm
import seaborn as sns
import torch

df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
if df.shape[0] == 2477:
    fast_sub = True
    fast_df = pd.DataFrame(([['00086460a852_study', 'negative 1 0 0 1 1'], 
                         ['000c9c05fd14_study', 'negative 1 0 0 1 1'], 
                         ['65761e66de9f_image', 'none 1 0 0 1 1'], 
                         ['51759b5579bc_image', 'none 1 0 0 1 1']]), 
                       columns=['id', 'PredictionString'])
else:
    fast_sub = False
fast_sub

True

# .dcm to .png

In [3]:
import numpy as np
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    
    return im

In [4]:
split = 'test'
save_dir = f'/kaggle/tmp/{split}/'

study_level_size = 768
image_level_size = 512

os.makedirs(save_dir, exist_ok=True)

# study level 용 이미지 생성 폴더
save_dir = f'/kaggle/tmp/{split}/study/'
os.makedirs(save_dir, exist_ok=True)

if fast_sub:
    xray = read_xray('../input/siim-covid19-detection/train/00086460a852/9e8302230c91/65761e66de9f.dcm')
    im = resize(xray, size=study_level_size)  
    study = '00086460a852' + '_study.png'
    im.save(os.path.join(save_dir, study))
    xray = read_xray('../input/siim-covid19-detection/train/000c9c05fd14/e555410bd2cd/51759b5579bc.dcm')
    im = resize(xray, size=study_level_size)  
    study = '000c9c05fd14' + '_study.png'
    im.save(os.path.join(save_dir, study))
else:   
    for dirname, _, filenames in tqdm(os.walk(f'../input/siim-covid19-detection/{split}')):
        for file in filenames:
            # set keep_ratio=True to have original aspect ratio
            xray = read_xray(os.path.join(dirname, file))
            im = resize(xray, size=study_level_size)  
            study = dirname.split('/')[-2] + '_study.png'
            im.save(os.path.join(save_dir, study))

In [5]:
# image_level 용 폴더 : 2 class 를 위해서는 생성해야 함.
image_id = []
dim0 = []
dim1 = []
splits = []
save_dir = f'/kaggle/tmp/{split}/image/'
os.makedirs(save_dir, exist_ok=True)

if fast_sub:
    xray = read_xray('../input/siim-covid19-detection/train/00086460a852/9e8302230c91/65761e66de9f.dcm')
    im = resize(xray, size=image_level_size)  
    im.save(os.path.join(save_dir,'65761e66de9f_image.png'))
    image_id.append('65761e66de9f.dcm'.replace('.dcm', ''))
    dim0.append(xray.shape[0])
    dim1.append(xray.shape[1])
    splits.append(split)
    xray = read_xray('../input/siim-covid19-detection/train/000c9c05fd14/e555410bd2cd/51759b5579bc.dcm')
    im = resize(xray, size=image_level_size)  
    im.save(os.path.join(save_dir, '51759b5579bc_image.png'))
    image_id.append('51759b5579bc.dcm'.replace('.dcm', ''))
    dim0.append(xray.shape[0])
    dim1.append(xray.shape[1])
    splits.append(split)
else:
    for dirname, _, filenames in tqdm(os.walk(f'../input/siim-covid19-detection/{split}')):
        for file in filenames:
            # set keep_ratio=True to have original aspect ratio
            xray = read_xray(os.path.join(dirname, file))
            im = resize(xray, size=image_level_size)  
            im.save(os.path.join(save_dir, file.replace('.dcm', '_image.png')))
            image_id.append(file.replace('.dcm', ''))
            dim0.append(xray.shape[0])
            dim1.append(xray.shape[1])
            splits.append(split)
meta = pd.DataFrame.from_dict({'image_id': image_id, 'dim0': dim0, 'dim1': dim1, 'split': splits})

# study predict

In [6]:
import numpy as np 
import pandas as pd

if fast_sub:
    df = fast_df.copy()
else:
    df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
id_laststr_list  = []
for i in range(df.shape[0]):
    id_laststr_list.append(df.loc[i,'id'][-1])
df['id_last_str'] = id_laststr_list

study_len = df[df['id_last_str'] == 'y'].shape[0]

In [7]:
!pip install /kaggle/input/kerasapplications -q
!pip install /kaggle/input/efficientnet-keras-source-code -q --no-deps

In [8]:
import os
import efficientnet.tfkeras as efn
import numpy as np
import pandas as pd
import tensorflow as tf

def auto_select_accelerator():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")

    return strategy


def build_decoder(with_labels=True, target_size=(300, 300), ext='jpg', tta=None):
    def decode(path):
        file_bytes = tf.io.read_file(path)
        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels=3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels=3)
        else:
            raise ValueError("Image extension not supported")

        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        if tta == None:
            return img
        elif tta == 'flip':
            return tf.image.flip_left_right(img)
        elif tta == 'flip_up_down':
            return tf.image.flip_up_down(img)

    def decode_with_labels(path, label):
        return decode(path), label

    return decode_with_labels if with_labels else decode


def build_augmenter(with_labels=True):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        return img

    def augment_with_labels(img, label):
        return augment(img), label

    return augment_with_labels if with_labels else augment


def build_dataset(paths, labels=None, bsize=32, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=True, repeat=True, shuffle=1024, 
                  cache_dir=""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)

    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)

    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)

    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)

    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO)

    return dset

#COMPETITION_NAME = "siim-cov19-test-img512-study-600"
strategy = auto_select_accelerator()
BATCH_SIZE = strategy.num_replicas_in_sync * 16

v1=False
v2=True
tta_study=True


if v1:
    IMSIZE = (224, 240, 260, 300, 380, 456, 528, 600, 512, 768)
    IMS = 7
elif v2:
    IMSIZE = (768,)
    IMS = 0
target_size = (IMSIZE[IMS], IMSIZE[IMS])
print(f"target_size : {target_size}")

#load_dir = f"/kaggle/input/{COMPETITION_NAME}/"
if fast_sub:
    sub_df = fast_df.copy()
else:
    sub_df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
sub_df = sub_df[:study_len]
test_paths = f'/kaggle/tmp/{split}/study/' + sub_df['id'] +'.png'

sub_df['negative'] = 0
sub_df['typical'] = 0
sub_df['indeterminate'] = 0
sub_df['atypical'] = 0

label_cols = sub_df.columns[2:]

print('build datset')
test_decoder = build_decoder(with_labels=False, 
                             target_size=target_size, # image_size
                             ext='png')
dtest = build_dataset(test_paths, 
                      bsize=BATCH_SIZE, 
                      repeat=False, 
                      shuffle=False, 
                      augment=False, 
                      cache=False,
                      decode_fn=test_decoder)

if tta_study:
    print('build tta_study datset')
    test_decoder_flip = build_decoder(with_labels=False, 
                                      target_size=target_size, 
                                      ext='png',
                                      tta='flip')

    dtest_flip = build_dataset(test_paths, 
                               bsize=BATCH_SIZE, 
                               repeat=False, 
                               shuffle=False, 
                               augment=False, 
                               cache=False,
                               decode_fn=test_decoder_flip)

Running on 1 replicas
target_size : (768, 768)
build datset
build tta_study datset


In [9]:
# study level 학습 모델
# efnV1_b7 현재 내가 직접 학습한 모델 0.428 
# efnV1_b7 기존에 학습되어 있던 모델 0.438
# efnV2_large : 0.441

import tensorflow as tf
import tensorflow_hub as tfhub

MODEL_ARCH = 'efficientnetv2-l-21k-ft1k'
# Get the Tensorflow Hub model URL
hub_type = 'feature_vector' # ['classification', 'feature_vector'] 중 선택
MODEL_ARCH_PATH = f"../input/efficientnetv2-tfhub-weight-files/tfhub_models/{MODEL_ARCH}/{hub_type}"

# Custom wrapper class to load the right pretrained weights explicitly from the local directory
class KerasLayerWrapper(tfhub.KerasLayer):
    def __init__(self, handle, **kargs):
        handle = tfhub.KerasLayer(tfhub.load(MODEL_ARCH_PATH))
        super().__init__(handle, **kargs)


if v1:
    print('\n efficientnet V1 for study_level \n')
    with strategy.scope():
        models = []
        models0 = tf.keras.models.load_model('../input/siim-covid19-efnb7-train-study/model0.h5')
        models1 = tf.keras.models.load_model('../input/siim-covid19-efnb7-train-study/model1.h5')
        models2 = tf.keras.models.load_model('../input/siim-covid19-efnb7-train-study/model2.h5')
        models3 = tf.keras.models.load_model('../input/siim-covid19-efnb7-train-study/model3.h5')
        models4 = tf.keras.models.load_model('../input/siim-covid19-efnb7-train-study/model4.h5')

        models.append(models0)
        models.append(models1)
        models.append(models2)
        models.append(models3)
        models.append(models4)
    result1 = sum([model.predict(dtest, verbose=1) for model in models]) / len(models)
    

if v2:
    print('\n efficientnetV2_large for study_level \n')
    MODEL_PATH = '/kaggle/input/siim-study-efnv2-model/study_efnv2_xl_model'
    with tf.device('/device:GPU:0'): # Specifies the device for ops created/executed in this context.
        models = []
        models0 = tf.keras.models.load_model(f"{MODEL_PATH}/model0.h5",
                                             custom_objects = {'KerasLayer':KerasLayerWrapper})
        models1 = tf.keras.models.load_model(f"{MODEL_PATH}/model1.h5",
                                             custom_objects = {'KerasLayer':KerasLayerWrapper})
        models2 = tf.keras.models.load_model(f"{MODEL_PATH}/model2.h5",
                                             custom_objects = {'KerasLayer':KerasLayerWrapper})
        models3 = tf.keras.models.load_model(f"{MODEL_PATH}/model3.h5",
                                             custom_objects = {'KerasLayer':KerasLayerWrapper})
        models4 = tf.keras.models.load_model(f"{MODEL_PATH}/model4.h5",
                                             custom_objects = {'KerasLayer':KerasLayerWrapper})
        models.append(models0)
        models.append(models1)
        models.append(models2)
        models.append(models3)
        models.append(models4)

            
    result1 = sum([model.predict(dtest, verbose=1) for model in models]) / len(models)        
    
if tta_study:
    print(f"try flip tta")
    result2 = sum([model.predict(dtest_flip, verbose=1) for model in models]) / len(models)        
    result = (result1 + result2)/2
    sub_df[label_cols] = result
else:
    sub_df[label_cols] = result1


 efficientnetV2_large for study_level 

1/1 [==============================] - 4s 4s/step
try flip tta
1/1 [==============================] - 0s 221ms/step


In [10]:
sub_df.columns = ['id', 'PredictionString1', 'negative', 'typical', 'indeterminate', 'atypical']
df = pd.merge(df, sub_df, on = 'id', how = 'left')

# study string

In [11]:
for i in range(study_len):
    negative = df.loc[i,'negative']
    typical = df.loc[i,'typical']
    indeterminate = df.loc[i,'indeterminate']
    atypical = df.loc[i,'atypical']
    df.loc[i, 'PredictionString'] = f'negative {negative} 0 0 1 1 typical {typical} 0 0 1 1 indeterminate {indeterminate} 0 0 1 1 atypical {atypical} 0 0 1 1'
df_study = df[['id', 'PredictionString']]

# df.to_csv('submission.csv',index=False)
# df

# 2 class
image level 에서 이미지 별로 none 이냐 아니냐를 판별하는 inference

In [12]:
if fast_sub:
    sub_df = fast_df.copy()
else:
    sub_df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
sub_df = sub_df[study_len:] # sub_df image_level 로 재정의
test_paths = f'/kaggle/tmp/{split}/image/' + sub_df['id'] +'.png'
sub_df['none'] = 0

label_cols = sub_df.columns[2]


v1 = True
v2 = False
tta_2class = True

if v1:
    IMSIZE = (224, 240, 260, 300, 380, 456, 528, 600, 512)
    IMS = 8
    print('efficientNetB7')
elif v2:
    IMSIZE = (512, 768,)
    IMS = 0
    print('efficientNetV2_large')
target_size = (IMSIZE[IMS], IMSIZE[IMS])

print(f"2 class target_size : {target_size}")

test_decoder = build_decoder(with_labels=False, 
                             target_size=target_size, # 512 size 
                             ext='png')
dtest = build_dataset(test_paths, 
                      bsize=BATCH_SIZE, 
                      repeat=False, 
                      shuffle=False, 
                      augment=False, 
                      cache=False,
                      decode_fn=test_decoder)
if tta_2class:
    print("load tta_2class dataset")
    test_decoder_flip = build_decoder(with_labels=False, 
                                      target_size=target_size, # 512 size 
                                      ext='png',
                                      tta='flip')
    dtest_flip = build_dataset(test_paths, 
                               bsize=BATCH_SIZE, 
                               repeat=False, 
                               shuffle=False, 
                               augment=False, 
                               cache=False,
                               decode_fn=test_decoder_flip)
    

if v1:
    print('V1 efficientNetB7 inference')
    with strategy.scope():
        models = []
        models0 = tf.keras.models.load_model('../input/efnb7-2class-new/kaggle/working/effnetB7_model/model0_2class.h5')
        models1 = tf.keras.models.load_model('../input/efnb7-2class-new/kaggle/working/effnetB7_model/model1_2class.h5')
        models2 = tf.keras.models.load_model('../input/efnb7-2class-new/kaggle/working/effnetB7_model/model2_2class.h5')
        models3 = tf.keras.models.load_model('../input/efnb7-2class-new/kaggle/working/effnetB7_model/model3_2class.h5')
        models4 = tf.keras.models.load_model('../input/efnb7-2class-new/kaggle/working/effnetB7_model/model4_2class.h5')

        models.append(models0)
        models.append(models1)
        models.append(models2)
        models.append(models3)
        models.append(models4)
    two_class_result1 = sum([model.predict(dtest, verbose=1) for model in models]) / len(models)

if v2:
    print('\n V2 efficientNetV2_large inference \n')
    # 'efficientnetv2-b1', 'efficientnetv2-l-21k-ft1k'
    MODEL_ARCH = 'efficientnetv2-l-21k-ft1k' # change by model
    # Get the Tensorflow Hub model URL
    hub_type = 'feature_vector' # ['classification', 'feature_vector'] 중 선택
    MODEL_ARCH_PATH = f"../input/efficientnetv2-tfhub-weight-files/tfhub_models/{MODEL_ARCH}/{hub_type}"

    # Custom wrapper class to load the right pretrained weights explicitly from the local directory
    class KerasLayerWrapper(tfhub.KerasLayer):
        def __init__(self, handle, **kargs):
            handle = tfhub.KerasLayer(tfhub.load(MODEL_ARCH_PATH))
            super().__init__(handle, **kargs)
        
    print('\n efficientnet V2 for 2 classifier \n')
    MODEL_PATH = '/kaggle/input/efnv2-2class-cutmix/kaggle/working/effnetV2_model' # efnV2_large
    with tf.device('/device:GPU:0'): # Specifies the device for ops created/executed in this context.
        models = []
        models0 = tf.keras.models.load_model(f"{MODEL_PATH}/model0_2class.h5",
                                             custom_objects = {'KerasLayer':KerasLayerWrapper})
        models1 = tf.keras.models.load_model(f"{MODEL_PATH}/model1_2class.h5",
                                             custom_objects = {'KerasLayer':KerasLayerWrapper})
        models2 = tf.keras.models.load_model(f"{MODEL_PATH}/model2_2class.h5",
                                             custom_objects = {'KerasLayer':KerasLayerWrapper})
        models3 = tf.keras.models.load_model(f"{MODEL_PATH}/model3_2class.h5",
                                             custom_objects = {'KerasLayer':KerasLayerWrapper})
        models4 = tf.keras.models.load_model(f"{MODEL_PATH}/model4_2class.h5",
                                             custom_objects = {'KerasLayer':KerasLayerWrapper})
        models.append(models0)
        models.append(models1)
        models.append(models2)
        models.append(models3)
        models.append(models4)
    two_class_result2 = sum([model.predict(dtest, verbose=1) for model in models]) / len(models)

    
if tta_2class:
    print(f"try flip tta (2class)")
    two_class_result1_tta = sum([model.predict(dtest_flip, verbose=1) for model in models]) / len(models)        
    
# 2021.07.31 현재 cutmix 구현된 efnb7 모델은 binary 예측이 아닌 categorical 예측이므로 prediction 값 변경이 필요 -> LB 오류

efficientNetB7
2 class target_size : (512, 512)
load tta_2class dataset
V1 efficientNetB7 inference
1/1 [==============================] - 4s 4s/step
try flip tta (2class)
1/1 [==============================] - 0s 124ms/step


In [13]:
if False:
    cutmix_model = False
    if cutmix_model:
        if v1:
            two_class_result1 = two_class_result1[:,1]
        elif v2:
            two_class_result2 = two_class_result2[:,1]

    if v1 and v2: # ensemble
        print('\n 2class ensemble \n')
        sub_df[label_cols] = (two_class_result1 + two_class_result2)/2

    if v1 and not v2: # only v1
        print('\n only EfficientNetB7 \n')    
        sub_df[label_cols] = two_class_result1

    if not v1 and v2: # only v2
        print('\n only EfficientNetV2_large \n')
        sub_df[label_cols] = two_class_result2

two_class_result = (two_class_result1 + two_class_result1_tta)/2
sub_df[label_cols] = two_class_result
        
df_2class = sub_df.reset_index(drop=True)
    
#     sub_df[label_cols] = sum([model.predict(dtest, verbose=1) for model in models]) / len(models)
#     df_2class = sub_df.reset_index(drop=True)

In [14]:
del models
del models0, models1, models2, models3, models4

In [15]:
from numba import cuda
import torch
cuda.select_device(0)
cuda.close()
cuda.select_device(0)

<weakproxy at 0x7f7047f488f0 to Device at 0x7f70481c7dd0>

# Image_Level Prediction

## yolov5 predict

In [16]:
from ensemble_boxes import weighted_boxes_fusion, nms

meta = meta[meta['split'] == 'test']
if fast_sub:
    test_df = fast_df.copy()
else:
    test_df = pd.read_csv('/kaggle/input/siim-covid19-detection/sample_submission.csv')
test_df = df[study_len:].reset_index(drop=True) 
meta['image_id'] = meta['image_id'] + '_image'
meta.columns = ['id', 'dim0', 'dim1', 'split']
test_df = pd.merge(test_df, meta, on = 'id', how = 'left')

In [17]:
train_test_split = False
tta_ensemble = False
kfold = True
WBF_ENSEMBLE = True

## 1. Yolo v5x kfold ensemble w/tta

In [18]:
# 단일 모델 또는 Yolo v5 에게 ensemble 을 맡길때.(YOLO 모델 내부에서 ensemble 가능) 

if train_test_split:
    if os.path.isdir('/kaggle/working/yolov5/runs/detect'):
        shutil.rmtree('/kaggle/working/yolov5/runs/detect')
    dim = 512 # 1024, 256, 'original'
    test_dir = f'/kaggle/tmp/{split}/image'
    # 여기만 바꿈
    # original yolo v5x path
    #f'/kaggle/input/siim-cov19-yolov5-train/yolov5/runs/train/exp/weights/best.pt'
    #weights_dir = f'/kaggle/input/yolo-fold/yolo_fold/fold_4/best.pt' 
    
    f0 = f'/kaggle/input/new-yolov5x/new_yolov5x/new_yolov5x_fold_model/fold_0/best.pt'
    f1 = f'/kaggle/input/new-yolov5x/new_yolov5x/new_yolov5x_fold_model/fold_1/best.pt'
    f2 = f'/kaggle/input/new-yolov5x/new_yolov5x/new_yolov5x_fold_model/fold_2/best.pt'
    f3 = f'/kaggle/input/new-yolov5x/new_yolov5x/new_yolov5x_fold_model/fold_3/best.pt'
    f4 = f'/kaggle/input/new-yolov5x/new_yolov5x/new_yolov5x_fold_model/fold_4/best.pt'
    
    if not os.path.isdir('/kaggle/working/yolov5'):
        shutil.copytree('/kaggle/input/yolov5-official-v31-dataset/yolov5', 
                        '/kaggle/working/yolov5')
    os.chdir('/kaggle/working/yolov5') # install dependencies

    import torch
    #from IPython.display import Image, clear_output  # to display images

    #clear_output()
    #print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))
    
    !python detect.py --weights $f0 $f1 $f2 $f3 $f4\
                      --img 512\
                      --conf 0.001\
                      --iou 0.5\
                      --source $test_dir\
                      --save-txt\
                      --save-conf\
                      --exist-ok\
                      --augment

    # default --conf 0.001
    # TTA : --augment (하기전에 LB 0.612)
    # AGNOSTIC_NMS : Classification 없이 물체의 바운딩 박스만을 찾고 싶을 때.

    def yolo2voc(image_height, image_width, bboxes):
        """
        yolo => [xmid, ymid, w, h] (normalized)
        voc  => [x1, y1, x2, y1]

        """ 
        bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int

        bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
        bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height

        bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
        bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]

        return bboxes

    image_ids = []
    PredictionStrings = []

    for file_path in tqdm(glob('runs/detect/exp/labels/*.txt')):
        image_id = file_path.split('/')[-1].split('.')[0]
        w, h = test_df.loc[test_df.id==image_id,['dim1', 'dim0']].values[0]
        f = open(file_path, 'r')
        data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
        data = data[:, [0, 5, 1, 2, 3, 4]]
        bboxes = list(np.round(np.concatenate((data[:, :2], np.round(yolo2voc(h, w, data[:, 2:]))), axis =1).reshape(-1), 12).astype(str))
        for idx in range(len(bboxes)):
            bboxes[idx] = str(int(float(bboxes[idx]))) if idx%6!=1 else bboxes[idx] # confidence 아닌 것만 float
        image_ids.append(image_id)
        PredictionStrings.append(' '.join(bboxes))

    pred_df = pd.DataFrame({'id':image_ids,
                            'PredictionString':PredictionStrings})

### 1-1) w/TTA + w/o TTA -> WBF ensemble

In [19]:
# detect
tta_ensemble = False

# tta 적용 및 적용하지 않은 yolov5 예측 값 사이의 앙상블
if tta_ensemble:
    tta_dir = '/kaggle/working/tta_dir/'
    if os.path.isdir(tta_dir):
        shutil.rmtree(tta_dir)
    os.makedirs(tta_dir)
                      
    meta.to_csv(tta_dir + f"meta.csv", index=0)
    pred_df.to_csv(tta_dir + f"pred_df.csv", index=0)                      
    
#     if os.path.isdir('/kaggle/working/yolov5/runs/detect'):
#         shutil.rmtree('/kaggle/working/yolov5/runs/detect')

    dim = 512 # 1024, 256, 'original'
    test_dir = f'/kaggle/tmp/{split}/image'
    # 여기만 바꿈
    # original yolo v5x path
    #f'/kaggle/input/siim-cov19-yolov5-train/yolov5/runs/train/exp/weights/best.pt'
    #weights_dir = f'/kaggle/input/yolo-fold/yolo_fold/fold_4/best.pt' 
    f0 = f'/kaggle/input/yolo-fold/yolo_fold/fold_0/best.pt'
    f1 = f'/kaggle/input/yolo-fold/yolo_fold/fold_1/best.pt'
    f2 = f'/kaggle/input/yolo-fold/yolo_fold/fold_2/best.pt'
    f3 = f'/kaggle/input/yolo-fold/yolo_fold/fold_3/best.pt'
    f4 = f'/kaggle/input/yolo-fold/yolo_fold/fold_4/best.pt'
    
    if not os.path.isdir('/kaggle/working/yolov5'):
        shutil.copytree('/kaggle/input/yolov5-official-v31-dataset/yolov5', 
                        '/kaggle/working/yolov5')
    os.chdir('/kaggle/working/yolov5') # install dependencies

    import torch
    #from IPython.display import Image, clear_output  # to display images

    #clear_output()
    #print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))


    !python detect.py --weights $f0 $f1 $f2 $f3 $f4\
                      --img 512\
                      --conf 0.001\
                      --iou 0.5\
                      --source $test_dir\
                      --save-txt\
                      --save-conf\
                      --exist-ok\
                      --name exp2

    image_ids = []
    PredictionStrings = []

    for file_path in tqdm(glob('runs/detect/exp2/labels/*.txt')):
        image_id = file_path.split('/')[-1].split('.')[0]
        w, h = test_df.loc[test_df.id==image_id,['dim1', 'dim0']].values[0]
        f = open(file_path, 'r')
        data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
        data = data[:, [0, 5, 1, 2, 3, 4]]
        bboxes = list(np.round(np.concatenate((data[:, :2], np.round(yolo2voc(h, w, data[:, 2:]))), axis =1).reshape(-1), 12).astype(str))
        for idx in range(len(bboxes)):
            bboxes[idx] = str(int(float(bboxes[idx]))) if idx%6!=1 else bboxes[idx] # confidence 아닌 것만 float
        image_ids.append(image_id)
        PredictionStrings.append(' '.join(bboxes))


    pred_df2 = pd.DataFrame({'id':image_ids,
                             'PredictionString':PredictionStrings})
    
    meta.to_csv(tta_dir + f"meta.csv", index=0)
    pred_df2.to_csv(tta_dir + f"pred_df2.csv", index=0)


In [20]:
# w/o tta + w/ tta -> WBF ensemble
if tta_ensemble: 
    height_dict = pd.read_csv(tta_dir + 'meta.csv')
    height_dict.columns = ['image_id','width','height','split']
    height_dict = height_dict[['image_id','width','height']]
    height_dict = height_dict.to_dict('records')
    
    fnl_dict = {}
    for ix, i in enumerate(height_dict):
        fnl_dict[i['image_id']] = [i['width'],i['height'],i['width'],i['height']]
        
    subs = [
        pd.read_csv(tta_dir + 'pred_df.csv'),
        pd.read_csv(tta_dir + 'pred_df2.csv')]
    
    def submission_decoder(df:pd.DataFrame) -> np.ndarray:
        info = df.values
        df_lst = []
        for i in info:
            pre_lst = i[1].split(' ')
            for j in range(0, len(pre_lst), 6):
                df_lst.append([i[0], int(pre_lst[j]), float(pre_lst[j+1]),
                               int(pre_lst[j+2]), int(pre_lst[j+3]),
                               int(pre_lst[j+4]), int(pre_lst[j+5]),
                               fnl_dict.get(i[0])[0], fnl_dict.get(i[0])[1]])
        return pd.DataFrame(df_lst, columns = ['image_id', 'class_id', 'score',
                                               'x_min','y_min','x_max','y_max',
                                               'width','height'])

    subs = [submission_decoder(subs[i]) for i in range(len(subs))]
    
    boxes_dict = {}
    scores_dict = {}
    labels_dict = {}
    whwh_dict = {}

    for i in tqdm(subs[0].image_id.unique()):
        if not i in boxes_dict.keys():
            boxes_dict[i] = []
            scores_dict[i] = []
            labels_dict[i] = []
            whwh_dict[i] = []

        size_ratio = fnl_dict.get(i)
        whwh_dict[i].append(size_ratio)
        tmp_df = [subs[x][subs[x]['image_id']==i] for x in range(len(subs))]

        for x in range(len(tmp_df)):
            boxes_dict[i].append(((tmp_df[x][['x_min','y_min','x_max','y_max']].values)/size_ratio).tolist())
            scores_dict[i].append(tmp_df[x]['score'].values.tolist())
            labels_dict[i].append(tmp_df[x]['class_id'].values.tolist())
            
    weights = [1] * 2
    iou_thr = 0.60
    skip_box_thr = 0.0
    sigma = 0.0

    fnl = {}

    for i in tqdm(boxes_dict.keys()):
        boxes, scores, labels = weighted_boxes_fusion(boxes_dict[i], 
                                                      scores_dict[i], 
                                                      labels_dict[i],
                                                      weights=weights,
                                                      iou_thr=iou_thr,
                                                      skip_box_thr=skip_box_thr)
        if not i in fnl.keys():
            fnl[i] = {'boxes':[],
                      'scores':[],
                      'labels':[]}
        fnl[i]['boxes'] = boxes * whwh_dict[i]
        fnl[i]['scores'] = scores
        fnl[i]['labels'] = labels
    
    pd_form = []
    
    for i in fnl.keys():
        b = fnl[i] # image_id
        for j in range(len(b['boxes'])):
            pd_form.append([i, 
                            int(b['labels'][j]), float(b['scores'][j]),
                            int(b['boxes'][j][0]), int(b['boxes'][j][1]),
                            int(b['boxes'][j][2]), int(b['boxes'][j][3])])
    final_df = pd.DataFrame(pd_form, columns = ['image_id', 'class_id', 'score',
                                                'x_min', 'y_min' ,'x_max', 'y_max'])
    final_df = final_df.drop_duplicates(keep = 'first')
    
    def submission_encoder(df:pd.DataFrame) -> np.ndarray:
        dct = {}
        for i in tqdm(df['image_id'].unique()):
            if not i in dct.keys():
                dct[i] = []
            tmp = df[df['image_id']==i].values
            for j in tmp:
                dct[i].append(int(j[1]))
                dct[i].append(float(j[2]))
                dct[i].append(int(j[3]))
                dct[i].append(int(j[4]))
                dct[i].append(int(j[5]))
                dct[i].append(int(j[6]))
            dct[i] = map(str, dct[i])
            dct[i] = ' '.join(dct[i])
        dct = [[k,v] for k, v in dct.items()]
        return pd.DataFrame(dct, columns = ['image_id', 'PredictionString']).reset_index(drop=True)

    tta_df = submission_encoder(final_df)
    
    pred_df = tta_df.copy()
    pred_df.columns = ['id','PredictionString']

## 2. KFOLD WBF Ensemble

In [21]:
# kfold ensemble by WBF (5번을 각각 돌려 나온 결과를 WBF)

if kfold:    
    kfold_result_dir = '/kaggle/working/kfold_result_dir'
    if os.path.isdir(kfold_result_dir):
        shutil.rmtree(kfold_result_dir)
    os.makedirs(kfold_result_dir)
        
    kfold_df = pd.DataFrame() # for weighted_box_fusion
    
    dim = 512 #1024, 256, 'original'
    test_dir = f'/kaggle/tmp/{split}/image'
    if os.path.isdir('/kaggle/working/yolov5/runs/detect'):
        shutil.rmtree('/kaggle/working/yolov5/runs/detect')
    
    if not os.path.isdir('/kaggle/working/yolov5'):
        shutil.copytree('/kaggle/input/yolov5-official-v31-dataset/yolov5', '/kaggle/working/yolov5')
    os.chdir('/kaggle/working/yolov5') # install dependencies
    
    
    for fold in range(5):
        print(f'##### {fold} inference #####')
        weights_dir = f'/kaggle/input/yolo-fold/yolo_fold/fold_{fold}/best.pt' # 여기만 바꿈
        name = f"exp_{fold}"
        !python detect.py --weights $weights_dir\
        --img 512\
        --conf 0.001\
        --iou 0.5\
        --source $test_dir\
        --name $name\
        --save-txt\
        --save-conf\
        --exist-ok\
        --augment

        # default --conf 0.001

        def yolo2voc(image_height, image_width, bboxes):
            """
            yolo => [xmid, ymid, w, h] (normalized)
            voc  => [x1, y1, x2, y1]

            """
            bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int

            bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
            bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height

            bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
            bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]
 
            return bboxes

        image_ids = []
        PredictionStrings = []
        
        path = f'runs/detect/exp_{fold}/labels/*.txt'

        for file_path in tqdm(glob(path)):
            image_id = file_path.split('/')[-1].split('.')[0]
            w, h = test_df.loc[test_df.id==image_id,['dim1', 'dim0']].values[0]
            f = open(file_path, 'r')
            data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
            data = data[:, [0, 5, 1, 2, 3, 4]]
            bboxes = list(np.round(np.concatenate((data[:, :2], np.round(yolo2voc(h, w, data[:, 2:]))), axis =1).reshape(-1), 12).astype(str))
            for idx in range(len(bboxes)):
                bboxes[idx] = str(int(float(bboxes[idx]))) if idx%6!=1 else bboxes[idx] # confidence 아닌 것만 float
            image_ids.append(image_id)
            PredictionStrings.append(' '.join(bboxes))
            
        pred_df = pd.DataFrame({'id':image_ids,
                                'PredictionString':PredictionStrings})
        
        pred_df.to_csv(kfold_result_dir + f"/kfold_{fold}_yolo.csv", index=0)
        kfold_df = kfold_df.append(pred_df)
        
    kfold_df.to_csv(kfold_result_dir + f"/kfold_df_yolo.csv", index=0)
    meta.to_csv(kfold_result_dir + f"/meta.csv", index=0)

##### 0 inference #####
Namespace(agnostic_nms=False, augment=True, classes=None, conf_thres=0.001, device='', exist_ok=True, img_size=512, iou_thres=0.5, name='exp_0', project='runs/detect', save_conf=True, save_txt=True, source='/kaggle/tmp/test/image', update=False, view_img=False, weights=['/kaggle/input/yolo-fold/yolo_fold/fold_0/best.pt'])
Fusing layers... 
image 1/2 /kaggle/tmp/test/image/51759b5579bc_image.png: 512x512 50 0. opacitys, Done. (0.163s)
image 2/2 /kaggle/tmp/test/image/65761e66de9f_image.png: 512x512 37 0. opacitys, Done. (0.139s)
Results saved to runs/detect/exp_0
2 labels saved to runs/detect/exp_0/labels
Done. (0.574s)


  0%|          | 0/2 [00:00<?, ?it/s]

##### 1 inference #####
Namespace(agnostic_nms=False, augment=True, classes=None, conf_thres=0.001, device='', exist_ok=True, img_size=512, iou_thres=0.5, name='exp_1', project='runs/detect', save_conf=True, save_txt=True, source='/kaggle/tmp/test/image', update=False, view_img=False, weights=['/kaggle/input/yolo-fold/yolo_fold/fold_1/best.pt'])
Fusing layers... 
image 1/2 /kaggle/tmp/test/image/51759b5579bc_image.png: 512x512 27 0. opacitys, Done. (0.147s)
image 2/2 /kaggle/tmp/test/image/65761e66de9f_image.png: 512x512 36 0. opacitys, Done. (0.141s)
Results saved to runs/detect/exp_1
2 labels saved to runs/detect/exp_1/labels
Done. (0.439s)


  0%|          | 0/2 [00:00<?, ?it/s]

##### 2 inference #####
Namespace(agnostic_nms=False, augment=True, classes=None, conf_thres=0.001, device='', exist_ok=True, img_size=512, iou_thres=0.5, name='exp_2', project='runs/detect', save_conf=True, save_txt=True, source='/kaggle/tmp/test/image', update=False, view_img=False, weights=['/kaggle/input/yolo-fold/yolo_fold/fold_2/best.pt'])
Fusing layers... 
image 1/2 /kaggle/tmp/test/image/51759b5579bc_image.png: 512x512 51 0. opacitys, Done. (0.178s)
image 2/2 /kaggle/tmp/test/image/65761e66de9f_image.png: 512x512 25 0. opacitys, Done. (0.143s)
Results saved to runs/detect/exp_2
2 labels saved to runs/detect/exp_2/labels
Done. (0.505s)


  0%|          | 0/2 [00:00<?, ?it/s]

##### 3 inference #####
Namespace(agnostic_nms=False, augment=True, classes=None, conf_thres=0.001, device='', exist_ok=True, img_size=512, iou_thres=0.5, name='exp_3', project='runs/detect', save_conf=True, save_txt=True, source='/kaggle/tmp/test/image', update=False, view_img=False, weights=['/kaggle/input/yolo-fold/yolo_fold/fold_3/best.pt'])
Fusing layers... 
image 1/2 /kaggle/tmp/test/image/51759b5579bc_image.png: 512x512 40 0. opacitys, Done. (0.141s)
image 2/2 /kaggle/tmp/test/image/65761e66de9f_image.png: 512x512 35 0. opacitys, Done. (0.182s)
Results saved to runs/detect/exp_3
2 labels saved to runs/detect/exp_3/labels
Done. (0.495s)


  0%|          | 0/2 [00:00<?, ?it/s]

##### 4 inference #####
Namespace(agnostic_nms=False, augment=True, classes=None, conf_thres=0.001, device='', exist_ok=True, img_size=512, iou_thres=0.5, name='exp_4', project='runs/detect', save_conf=True, save_txt=True, source='/kaggle/tmp/test/image', update=False, view_img=False, weights=['/kaggle/input/yolo-fold/yolo_fold/fold_4/best.pt'])
Fusing layers... 
image 1/2 /kaggle/tmp/test/image/51759b5579bc_image.png: 512x512 30 0. opacitys, Done. (0.156s)
image 2/2 /kaggle/tmp/test/image/65761e66de9f_image.png: 512x512 41 0. opacitys, Done. (0.149s)
Results saved to runs/detect/exp_4
2 labels saved to runs/detect/exp_4/labels
Done. (0.473s)


  0%|          | 0/2 [00:00<?, ?it/s]

In [22]:
# fold ensemble - WBF
if WBF_ENSEMBLE:    
    height_dict = pd.read_csv('/kaggle/working/kfold_result_dir/meta.csv')
    height_dict.columns = ['image_id','width','height','split']
    height_dict = height_dict[['image_id','width','height']]
    height_dict = height_dict.to_dict('records')

    fnl_dict = {}
    for ix, i in enumerate(height_dict):
        fnl_dict[i['image_id']] = [i['width'],i['height'],i['width'],i['height']]
    
    subs = [pd.read_csv('/kaggle/working/kfold_result_dir/kfold_0_yolo.csv'),
            pd.read_csv('/kaggle/working/kfold_result_dir/kfold_1_yolo.csv'),
            pd.read_csv('/kaggle/working/kfold_result_dir/kfold_3_yolo.csv'),
            pd.read_csv('/kaggle/working/kfold_result_dir/kfold_2_yolo.csv'),
            pd.read_csv('/kaggle/working/kfold_result_dir/kfold_4_yolo.csv')]
    
    def submission_decoder(df:pd.DataFrame) -> np.ndarray:
        info = df.values
        df_lst = []
        for i in info:
            pre_lst = i[1].split(' ')
            for j in range(0, len(pre_lst), 6):
                df_lst.append([i[0], 
                               int(pre_lst[j]) if pre_lst[j] == '0' else 0,
                               float(pre_lst[j+1]),
                               int(pre_lst[j+2]), int(pre_lst[j+3]),
                               int(pre_lst[j+4]), int(pre_lst[j+5]),
                               fnl_dict.get(i[0])[0], fnl_dict.get(i[0])[1]])
        return pd.DataFrame(df_lst, columns = ['image_id', 'class_id', 'score',
                                               'x_min','y_min','x_max','y_max',
                                               'width','height'])

    subs = [submission_decoder(subs[i]) for i in range(len(subs))]
    
    boxes_dict = {}
    scores_dict = {}
    labels_dict = {}
    whwh_dict = {}

    for i in tqdm(subs[0].image_id.unique()):
        if not i in boxes_dict.keys():
            boxes_dict[i] = []
            scores_dict[i] = []
            labels_dict[i] = []
            whwh_dict[i] = []

        size_ratio = fnl_dict.get(i)
        whwh_dict[i].append(size_ratio)
        tmp_df = [subs[x][subs[x]['image_id']==i] for x in range(len(subs))]

        for x in range(len(tmp_df)):
            boxes_dict[i].append(((tmp_df[x][['x_min','y_min','x_max','y_max']].values)/size_ratio).tolist())
            scores_dict[i].append(tmp_df[x]['score'].values.tolist())
            labels_dict[i].append(tmp_df[x]['class_id'].values.tolist())
            
    weights = [1] * 6
    iou_thr = 0.60
    skip_box_thr = 0.0
    sigma = 0.0

    fnl = {}

    for i in tqdm(boxes_dict.keys()):
        boxes, scores, labels = weighted_boxes_fusion(boxes_dict[i], 
                                                      scores_dict[i], 
                                                      labels_dict[i],
                                                      weights=weights,
                                                      iou_thr=iou_thr,
                                                      skip_box_thr=skip_box_thr)
        if not i in fnl.keys():
            fnl[i] = {'boxes':[],
                      'scores':[],
                      'labels':[]}
        fnl[i]['boxes'] = boxes * whwh_dict[i]
        fnl[i]['scores'] = scores
        fnl[i]['labels'] = labels
    
    pd_form = []
    for i in fnl.keys():
        b = fnl[i] # image_id
        for j in range(len(b['boxes'])):
            pd_form.append([i, 
                            int(b['labels'][j]), float(b['scores'][j]),
                            int(b['boxes'][j][0]), int(b['boxes'][j][1]),
                            int(b['boxes'][j][2]), int(b['boxes'][j][3])])
    final_df = pd.DataFrame(pd_form, columns = ['image_id', 'class_id', 'score',
                                                'x_min', 'y_min' ,'x_max', 'y_max'])
    final_df = final_df.drop_duplicates(keep = 'first')
    
    def submission_encoder(df:pd.DataFrame) -> np.ndarray:
        dct = {}
        for i in tqdm(df['image_id'].unique()):
            if not i in dct.keys():
                dct[i] = []
            tmp = df[df['image_id']==i].values
            for j in tmp:
                dct[i].append(int(j[1]))
                dct[i].append(float(j[2]))
                dct[i].append(int(j[3]))
                dct[i].append(int(j[4]))
                dct[i].append(int(j[5]))
                dct[i].append(int(j[6]))
            dct[i] = map(str, dct[i])
            dct[i] = ' '.join(dct[i])
        dct = [[k,v] for k, v in dct.items()]
        return pd.DataFrame(dct, columns = ['image_id', 'PredictionString']).reset_index(drop=True)

    wbf_df = submission_encoder(final_df)
    

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [23]:
if WBF_ENSEMBLE:
    pred_df = wbf_df.copy()
    pred_df.columns = ['id','PredictionString']

In [24]:
# os.chdir('/')
# !zip /kaggle/working/kfold_result_dir.zip /kaggle/working/kfold_result_dir/*

### Last process

In [25]:
# image_level label 제작
test_df = test_df.drop(['PredictionString'], axis=1)
sub_df = pd.merge(test_df, pred_df, on = 'id', how = 'left').fillna("none 1 0 0 1 1")
sub_df = sub_df[['id', 'PredictionString']]
for i in range(sub_df.shape[0]): # only image_level df
    if sub_df.loc[i,'PredictionString'] == "none 1 0 0 1 1":
        continue
    sub_df_split = sub_df.loc[i,'PredictionString'].split() # box 가 있을때 label 값의 형식 변환
    sub_df_list = []
    for j in range(int(len(sub_df_split) / 6)):
        sub_df_list.append('opacity')
        sub_df_list.append(sub_df_split[6 * j + 1])
        sub_df_list.append(sub_df_split[6 * j + 2])
        sub_df_list.append(sub_df_split[6 * j + 3])
        sub_df_list.append(sub_df_split[6 * j + 4])
        sub_df_list.append(sub_df_split[6 * j + 5])
    sub_df.loc[i,'PredictionString'] = ' '.join(sub_df_list)
sub_df['none'] = df_2class['none']

In [26]:
# 2class 제작 : none class 확률에 기반하여 none label 추가 (none confidence 0 0 1 1)
for i in range(sub_df.shape[0]):
    if sub_df.loc[i,'PredictionString'] != 'none 1 0 0 1 1':
        sub_df.loc[i,'PredictionString'] = sub_df.loc[i,'PredictionString'] + ' none ' + str(sub_df.loc[i,'none']) + ' 0 0 1 1'

In [27]:
sub_df = sub_df[['id', 'PredictionString']]   
df_study = df_study[:study_len]
df_study = df_study.append(sub_df).reset_index(drop=True)
df_study.to_csv('/kaggle/working/submission.csv',index = False)  
shutil.rmtree('/kaggle/working/yolov5')

In [28]:
# dicom 변환시 사이즈 조정 -> 성능에 영향?

In [29]:
# 위에서의 sub_df : image_level 에 대해서 예측값 + none filling 2개의 컬럼 보유 df
# 아래 코드에서는 study_level 과 2 class 에 대해서만 반영하여 csv 파일 추출
# but, 2class 는 image_level 이 inference 된 후에 추가적으로 반영하는 것.
# 즉, 순서 : study_level + image_level + 2 classifier
# 그러면, study_level + image_level 부터 시도해보자.

In [30]:
df_study

,id,PredictionString
0,00086460a852_study,negative 0.07604188472032547 0 0 1 1 typical 0...
1,000c9c05fd14_study,negative 0.009672800078988075 0 0 1 1 typical ...
2,65761e66de9f_image,opacity 0.14389634132385254 1906 812 2401 1474...
3,51759b5579bc_image,opacity 0.1282227337360382 179 1876 1050 2524 ...
